In [1]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC 

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Read Test and Train file
trainfile = r'RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

testfile = r'RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(test_data.shape)
print(train_data.head())  
print(test_data.head())


(65000, 596)
(173836, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16  

In [3]:
#Separate Target data column

X_Train = train_data

Y_Train=X_Train["QuoteConversion_Flag"]

X_Train1=X_Train.iloc[:, :-1].copy()

# Split train data set

X_train1, X_test1, y_train, y_test = train_test_split(X_Train1, Y_Train, test_size = .25, 
                                                    stratify=Y_Train,random_state = 1)


In [31]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 

clf = DecisionTreeClassifier()
clf.fit(X_train1,y_train)
clf_predict=clf.predict(X_test1)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))


accuracy Score (training) for Decision Tree:0.882646
Confusion Matrix for Decision Tree
[[12181  1003]
 [  904  2162]]


In [32]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,5)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train1, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train1,y_train)
clf_predict = clf.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testing) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

#get cross-validation report
clf_cv_score = cross_val_score(clf, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


{'min_samples_split': 70, 'max_depth': 11}
accuracy Score (testing) after hypertuning for Decision Tree:0.914031
Confusion Matrix after hypertuning for Decision Tree
[[12881   303]
 [ 1094  1972]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     13184
           1       0.87      0.64      0.74      3066

    accuracy                           0.91     16250
   macro avg       0.89      0.81      0.84     16250
weighted avg       0.91      0.91      0.91     16250

=== All AUC Scores ===
[0.94898357 0.94454052 0.9327887  0.94344179 0.94190265 0.94308374
 0.94279077 0.94108654 0.93954076 0.93917839]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9417337430569882


In [8]:
# Process test data without geographical field column
X_TEST1= test_data.drop(["GeographicField64"], axis=1)

In [9]:
#prediction on test file based on decision tree classifier
clf_predict1 = clf.predict(X_TEST1)

In [10]:
# Generate CSV for kaggle Score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':clf_predict1})
X1.to_csv('DecisionTree_1.csv')

In [11]:
#Construct Random Forest Model

rfc = RandomForestClassifier()
rfc.fit(X_train1, y_train)
rfc_predict=rfc.predict(X_test1)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test1,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))

accuracy Score (training) for RandomForest:0.905785
Confusion Matrix for Random Forest:
[[12967   217]
 [ 1314  1752]]


In [12]:
#prediction on test file based on random forest classifier
rfc_predict1 = rfc.predict(X_TEST1)

In [13]:
# Generate CSV for kaggle Score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':rfc_predict1})
X1.to_csv('Random_Forest1.csv')

In [14]:
#Construct MultiLayer Perceptron Model
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=200)
mlp.fit(X_train1, y_train)
mlp_predict=mlp.predict(X_test1)
print("accuracy Score (training) for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test1,y_test)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test,mlp_predict))


accuracy Score (training) for MultiLayer Perceptron:0.705846
Confusion Matrix for MultiLayer Perceptron:
[[9212 3972]
 [ 808 2258]]


In [15]:
#prediction on test file based on Multiplayer Perceptron classifier
mlp_predict1 = mlp.predict(X_TEST1)

In [16]:
# Generate CSV for kaggle Score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':mlp_predict1})
X1.to_csv('MLP1.csv')

In [18]:
#Construct K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_train1, y_train)
neigh_predict=neigh.predict(X_test1)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test1,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,neigh_predict))


accuracy Score (training) for KNeighborsClassifier:0.763877
Confusion Matrix for KNeighborsClassifier:
[[12074  1110]
 [ 2727   339]]


In [21]:
#prediction on test file based on K-Nearest Neighbor Model
neigh_predict1 = neigh.predict(X_TEST1)

In [22]:
# Generate CSV for kaggle Score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':neigh_predict1})
X1.to_csv('KNN.csv')

In [23]:
#Construct Linear Support Vector Machine Model
from sklearn.svm import LinearSVC 
linsvm = LinearSVC(max_iter=1000) 
linsvm.fit(X_train1, y_train) 
linsvm_predict=linsvm.predict(X_test1) 
print("accuracy Score (training) for Linear SVM Classifier:{0:6f}".format(linsvm.score(X_test1,y_test))) 
print("Confusion Matrix for Linear SVM Classifier:") 
print(confusion_matrix(y_test,linsvm_predict))


accuracy Score (training) for Linear SVM Classifier:0.810892
Confusion Matrix for Linear SVM Classifier:
[[13164    20]
 [ 3053    13]]


In [24]:
#prediction on test file based on K-Nearest Neighbor Model
linsvm_predict1 = linsvm.predict(X_TEST1)

In [25]:
# Generate CSV for kaggle Score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':linsvm_predict1})
X1.to_csv('linsvm.csv')

In [26]:
#Construct Gradient Boosting model

search_grid={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train1, y_train)
abc_predict=abc.predict(X_test1)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_test1,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train1, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train1,y_train)
abc_predict = abc.predict(X_test1)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))
abc_cv_score = cross_val_score(abc, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())

accuracy Score (training) for Boosting:0.922031
Confusion Matrix for boosting:
[[13037   147]
 [ 1120  1946]]
{'n_estimators': 20, 'learning_rate': 0.1}
accuracy Score (training) after hypertuning for Boosting:0.891631
Confusion Matrix after hypertuning for Boosting:
[[13171    13]
 [ 1748  1318]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     13184
           1       0.99      0.43      0.60      3066

    accuracy                           0.89     16250
   macro avg       0.94      0.71      0.77     16250
weighted avg       0.90      0.89      0.87     16250

=== All AUC Scores ===
[0.93871116 0.93683594 0.92861259 0.93582894 0.93439853 0.94255455
 0.93600561 0.93878909 0.93413703 0.92518551]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.935105895274582


In [27]:
#prediction on test file based on K-Nearest Neighbor Model
abc_predict1 = abc.predict(X_TEST1)

In [28]:
# Generate CSV for kaggle Score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':abc_predict1})
X1.to_csv('GBM.csv')

# ***SMOTE***

In [5]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(X_train1, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 39554, 1: 9196})
Resampled dataset shape Counter({0: 39554, 1: 19777})


# ***ENSEMBLE METHODS STACKING***

In [9]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_TEST1,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.69736399]
    fold  1:  [0.69433021]
    fold  2:  [0.69783591]
    fold  3:  [0.68965750]
    ----
    MEAN:     [0.69479690] + [0.00325794]
    FULL:     [0.69479699]

model  1:     [MLPClassifier]
    fold  0:  [0.86819929]
    fold  1:  [0.83118722]
    fold  2:  [0.75372480]
    fold  3:  [0.82571467]
    ----
    MEAN:     [0.81970649] + [0.04145190]
    FULL:     [0.81970639]

model  2:     [LinearSVC]
    fold  0:  [0.57722645]
    fold  1:  [0.37409829]
    fold  2:  [0.55524843]
    fold  3:  [0.68797195]
    ----
    MEAN:     [0.54863628] + [0.11262666]
    FULL:     [0.54863394]

model  3:     [RandomForestCla

In [10]:
#Construct Random Forest Model

model = RandomForestClassifier()
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)


In [21]:
#Hyperparameter tuning for random forest classifier
rfc= RandomForestClassifier()
rand_parameters={'min_samples_leaf' : range(20,200,10),'max_depth': 
            range(5,20,3),'max_features':['auto', 'sqrt'],'n_estimators':[30,40,50,60]}
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=25,cv=5)
rfc_random.fit(S_Train, y_res)
random_parm_rfc=rfc_random.best_params_
print(random_parm_rfc)

#Construct Random Forest with best parameters
rfc= RandomForestClassifier(**random_parm_rfc)
rfc.fit(S_Train,y_res)
rfc_predict = rfc.predict(S_Test)


{'n_estimators': 50, 'min_samples_leaf': 190, 'max_features': 'sqrt', 'max_depth': 14}


In [22]:
# CSV File for kaggle score 
X1 = pd.DataFrame({'QuoteNumber':X_TEST1.QuoteNumber,'QuoteConversionFlag':rfc_predict})
X1.to_csv('RFC_SMOTE_1.csv')